In [19]:
# desc
# toc
# meta title
# meta desc
# meta keywords

In [4]:
import csv

with open("Book1.csv", "r") as csvfile:
    reader = csv.reader(csvfile, delimiter=","); titles = []
    for row in reader:
        title = row[0]; titles.append(title)

In [5]:
import os
import dotenv

dotenv.load_dotenv()
api_key = os.environ.get("GROQ_API_KEY")

if not api_key:
    raise ValueError("GROQ_API_KEY environment variable not set")

In [22]:
first2titles = titles[:2]

In [2]:
import json
from groq import Groq

def get_llama_assistance(prompt):
    client = Groq(api_key=api_key)
    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
            {
                "role": "assistant",
                "content": "",
            },
        ],
        temperature=1.4,
        max_tokens=8192,
        top_p=1,
        stream=True,
        stop=None,
    )
    response_text = ""
    for chunk in completion:
        response_text += chunk.choices[0].delta.content or ""
    return response_text


def generate_content(titles):
    data = dict()  # Explicitly create an empty dictionary
    for title in titles:
        content_dict = {"title": title}
        # Generate content for each item
        content_dict["description"] = get_llama_assistance(
            f"Write a compelling description for the title (Just Give Direct Answers): {title}"
        )
        content_dict["table_of_contents"] = get_llama_assistance(
            f"Generate a table of contents for content related to (Just Give Direct Answers): {title}"
        )
        content_dict["meta_title"] = get_llama_assistance(
            f"Create a meta title for content about (Just Give Direct Answers): {title}"
        )
        content_dict["meta_description"] = get_llama_assistance(
            f"Write a meta description summarizing content on (Just Give Direct Answers): {title}"
        )
        content_dict["meta_keywords"] = get_llama_assistance(
            f"Suggest relevant meta keywords for content related to (Just Give Direct Answers): {title}"
        )
        data[title] = content_dict  # Store content with title as key
    print(data)  

    with open("generated_content.json", "w") as outfile:
        json.dump(data, outfile)

In [ ]:
generate_content(first2titles)
print("Content generation complete! Check generated_content.json file.")